# Курсовая работа: Denoising автоэнкодеры (полносвязный и сверточный)

## Цель
Исследовать качество восстановления изображений с помощью двух вариантов denoising автоэнкодера:
1. Сверточный denoising автоэнкодер
2. Полносвязный denoising автоэнкодер

## Описание
- Denoising автоэнкодер восстанавливает чистые изображения из зашумленных входов
- Сравниваются полносвязная и сверточная архитектуры при одинаковых условиях обучения


In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("=" * 50)
print("Проверка PyTorch и GPU")
print("=" * 50)
print(f"PyTorch версия: {torch.__version__}")
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA версия: {torch.version.cuda}")
    print("✅ GPU готов к использованию!")
else:
    print("⚠️ GPU не найден - будет использоваться CPU")
print("=" * 50)
print()

from config import SEED, DEVICE, DENOISING_CONFIG
from utils import (
    set_seed, get_mnist_loader, get_fashion_mnist_loader,
    visualize_denoising_results,
    count_parameters, add_noise
)
from denoising_ae import ConvDenoisingAE, FCDenoisingAE, train_denoising_ae, evaluate_denoising_ae

# Настройка визуализации
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

# Фиксация seed
set_seed(SEED)
print(f"Используется устройство: {DEVICE}")
print(f"Seed установлен: {SEED}")


Проверка PyTorch и GPU
PyTorch версия: 2.5.1+cu121
CUDA доступна: True
GPU: NVIDIA GeForce RTX 3080
CUDA версия: 12.1
✅ GPU готов к использованию!

Используется устройство: cuda
Seed установлен: 42


Denoising Автоэнкодер

### Цель
Обучить автоэнкодер для восстановления зашумленных изображений.

### Описание
Сравниваем полносвязный и сверточный варианты автоэнкодера для задачи denoising.


In [2]:
# Загрузка данных для denoising автоэнкодера
print("Загрузка данных MNIST для denoising автоэнкодера...")
train_loader_denoising = get_mnist_loader(
    batch_size=DENOISING_CONFIG['batch_size'],
    train=True
)

test_loader_denoising = get_mnist_loader(
    batch_size=DENOISING_CONFIG['batch_size'],
    train=False
)

print(f"Размер батча: {DENOISING_CONFIG['batch_size']}")
print(f"Коэффициент шума: {DENOISING_CONFIG['noise_factor']}")


Загрузка данных MNIST для denoising автоэнкодера...
Размер батча: 256
Коэффициент шума: 0.5


### Сверточный автоэнкодер


In [3]:
# Создание сверточного denoising автоэнкодера
model_conv_ae = ConvDenoisingAE(input_size=28, latent_dim=128)

print(f"Количество параметров: {count_parameters(model_conv_ae):,}")
print("Архитектура: сверточные слои для энкодера и декодера")


Количество параметров: 1,302,657
Архитектура: сверточные слои для энкодера и декодера


In [ ]:
# Обучение сверточного автоэнкодера
print("Начинаем обучение сверточного автоэнкодера...")
history_conv = train_denoising_ae(
    model_conv_ae,
    train_loader_denoising,
    test_loader_denoising,
    epochs=DENOISING_CONFIG['epochs'],
    lr=DENOISING_CONFIG['lr'],
    noise_factor=DENOISING_CONFIG['noise_factor'],
    weight_decay=DENOISING_CONFIG['weight_decay']
)


Начинаем обучение сверточного автоэнкодера...
Модель имеет 1,302,657 параметров
Начинаем обучение на cuda...
Количество эпох: 100, Learning rate: 0.001, Noise factor: 0.5


Epoch 5/100: 100%|██████████| 235/235 [00:06<00:00, 34.42it/s, loss=0.0116]



Эпоха 5/100:
  Потеря на обучении: 0.0119
  Потеря на тесте: 0.0116
  PSNR: 19.30 dB (чем выше, тем лучше)
  SSIM: 0.9036 (чем ближе к 1, тем лучше)


Epoch 10/100: 100%|██████████| 235/235 [00:06<00:00, 35.86it/s, loss=0.0105]



Эпоха 10/100:
  Потеря на обучении: 0.0111
  Потеря на тесте: 0.0110
  PSNR: 19.52 dB (чем выше, тем лучше)
  SSIM: 0.9082 (чем ближе к 1, тем лучше)


...

Epoch 70/100: 100%|██████████| 235/235 [00:06<00:00, 34.09it/s, loss=0.0101]



Эпоха 70/100:
  Потеря на обучении: 0.0099
  Потеря на тесте: 0.0099
  PSNR: 19.97 dB (чем выше, тем лучше)
  SSIM: 0.9160 (чем ближе к 1, тем лучше)


Epoch 75/100: 100%|██████████| 235/235 [00:06<00:00, 34.87it/s, loss=0.0112]



Эпоха 75/100:
  Потеря на обучении: 0.0098
  Потеря на тесте: 0.0098
  PSNR: 19.98 dB (чем выше, тем лучше)
  SSIM: 0.9164 (чем ближе к 1, тем лучше)


Epoch 77/100: 100%|██████████| 235/235 [00:06<00:00, 34.19it/s, loss=0.0097]


In [ ]:
# Визуализация кривых обучения (сверточный)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Потеря
axes[0].plot(history_conv['epoch'], history_conv['train_loss'], 'b-', label='Обучение', linewidth=2)
axes[0].plot(history_conv['epoch'], history_conv['test_loss'], 'r-', label='Тест', linewidth=2)
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Кривая потерь (сверточный)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PSNR
axes[1].plot(history_conv['epoch'], history_conv['test_psnr'], 'g-', linewidth=2)
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('PSNR (dB)')
axes[1].set_title('PSNR на тестовой выборке')
axes[1].grid(True, alpha=0.3)

# SSIM
axes[2].plot(history_conv['epoch'], history_conv['test_ssim'], 'm-', linewidth=2)
axes[2].set_xlabel('Эпоха')
axes[2].set_ylabel('SSIM')
axes[2].set_title('SSIM на тестовой выборке')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Визуализация примеров восстановления (сверточный)
print("Примеры восстановления изображений (сверточный автоэнкодер):")
fig = visualize_denoising_results(
    model_conv_ae,
    test_loader_denoising,
    DEVICE,
    num_samples=8,
    noise_factor=DENOISING_CONFIG['noise_factor']
)
plt.show()


### Полносвязный автоэнкодер


In [ ]:
# Создание полносвязного denoising автоэнкодера
model_fc_ae = FCDenoisingAE(input_size=28, hidden_dims=[512, 256, 128])

print(f"Количество параметров: {count_parameters(model_fc_ae):,}")
print("Архитектура: только полносвязные слои")


In [ ]:
# Обучение полносвязного автоэнкодера
print("Начинаем обучение полносвязного автоэнкодера...")
history_fc = train_denoising_ae(
    model_fc_ae,
    train_loader_denoising,
    test_loader_denoising,
    epochs=DENOISING_CONFIG['epochs'],
    lr=DENOISING_CONFIG['lr'],
    noise_factor=DENOISING_CONFIG['noise_factor'],
    weight_decay=DENOISING_CONFIG['weight_decay']
)


In [ ]:
# Визуализация кривых обучения (полносвязный)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Потеря
axes[0].plot(history_fc['epoch'], history_fc['train_loss'], 'b-', label='Обучение', linewidth=2)
axes[0].plot(history_fc['epoch'], history_fc['test_loss'], 'r-', label='Тест', linewidth=2)
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Кривая потерь (полносвязный)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PSNR
axes[1].plot(history_fc['epoch'], history_fc['test_psnr'], 'g-', linewidth=2)
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('PSNR (dB)')
axes[1].set_title('PSNR на тестовой выборке')
axes[1].grid(True, alpha=0.3)

# SSIM
axes[2].plot(history_fc['epoch'], history_fc['test_ssim'], 'm-', linewidth=2)
axes[2].set_xlabel('Эпоха')
axes[2].set_ylabel('SSIM')
axes[2].set_title('SSIM на тестовой выборке')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Визуализация примеров восстановления (полносвязный)
print("Примеры восстановления изображений (полносвязный автоэнкодер):")
fig = visualize_denoising_results(
    model_fc_ae,
    test_loader_denoising,
    DEVICE,
    num_samples=8,
    noise_factor=DENOISING_CONFIG['noise_factor']
)
plt.show()


### Сравнение моделей


In [ ]:
# Сравнение метрик на тестовой выборке
print("="*60)
print("Сравнение моделей на тестовой выборке")
print("="*60)

# Оценка сверточного
print("\nСверточный автоэнкодер:")
metrics_conv = evaluate_denoising_ae(
    model_conv_ae,
    test_loader_denoising,
    noise_factor=DENOISING_CONFIG['noise_factor']
)

# Оценка полносвязного
print("\nПолносвязный автоэнкодер:")
metrics_fc = evaluate_denoising_ae(
    model_fc_ae,
    test_loader_denoising,
    noise_factor=DENOISING_CONFIG['noise_factor']
)

# Сводная таблица
print("\n" + "="*60)
print("Сводная таблица результатов:")
print("="*60)
print(f"{'Модель':<20} {'MSE':<12} {'PSNR (dB)':<12} {'SSIM':<12}")
print("-"*60)
print(f"{'Сверточный':<20} {metrics_conv['mse']:<12.6f} {metrics_conv['psnr']:<12.2f} {metrics_conv['ssim']:<12.4f}")
print(f"{'Полносвязный':<20} {metrics_fc['mse']:<12.6f} {metrics_fc['psnr']:<12.2f} {metrics_fc['ssim']:<12.4f}")
print("="*60)


In [ ]:
# Визуальное сравнение моделей
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Сравнение потерь
axes[0, 0].plot(history_conv['epoch'], history_conv['test_loss'], 'b-', label='Сверточный', linewidth=2)
axes[0, 0].plot(history_fc['epoch'], history_fc['test_loss'], 'r-', label='Полносвязный', linewidth=2)
axes[0, 0].set_xlabel('Эпоха')
axes[0, 0].set_ylabel('MSE Loss')
axes[0, 0].set_title('Сравнение потерь')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Сравнение PSNR
axes[0, 1].plot(history_conv['epoch'], history_conv['test_psnr'], 'b-', label='Сверточный', linewidth=2)
axes[0, 1].plot(history_fc['epoch'], history_fc['test_psnr'], 'r-', label='Полносвязный', linewidth=2)
axes[0, 1].set_xlabel('Эпоха')
axes[0, 1].set_ylabel('PSNR (dB)')
axes[0, 1].set_title('Сравнение PSNR')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Сравнение SSIM
axes[1, 0].plot(history_conv['epoch'], history_conv['test_ssim'], 'b-', label='Сверточный', linewidth=2)
axes[1, 0].plot(history_fc['epoch'], history_fc['test_ssim'], 'r-', label='Полносвязный', linewidth=2)
axes[1, 0].set_xlabel('Эпоха')
axes[1, 0].set_ylabel('SSIM')
axes[1, 0].set_title('Сравнение SSIM')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Финальные метрики (барчарт)
models = ['Сверточный', 'Полносвязный']
psnr_values = [metrics_conv['psnr'], metrics_fc['psnr']]
ssim_values = [metrics_conv['ssim'], metrics_fc['ssim']]

x = np.arange(len(models))
width = 0.35

axes[1, 1].bar(x - width/2, psnr_values, width, label='PSNR', alpha=0.8)
axes[1, 1].set_ylabel('PSNR (dB)')
axes[1, 1].set_title('Финальные метрики')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(models)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Добавляем значения SSIM на второй оси
ax2 = axes[1, 1].twinx()
ax2.bar(x + width/2, ssim_values, width, label='SSIM', color='orange', alpha=0.8)
ax2.set_ylabel('SSIM')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()
